<a href="https://colab.research.google.com/github/BluRaf/tree-classify/blob/master/Keras0_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install tensorflow-gpu tensorflow numpy

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [22]:
data_dir = '/content/drive/MyDrive/studia/sem5/pk/dataset'
size = (224, 224)

In [53]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=size
  )


Found 1647 files belonging to 5 classes.
Using 1153 files for training.


In [59]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=size)

val_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

test_ds = validation_ds.take(0.5 * val_ds_size)
validation_ds = validation_ds.skip(0.5 * val_ds_size)

Found 1647 files belonging to 5 classes.
Using 494 files for validation.


In [60]:
class_names = train_ds.class_names
print(class_names)

['akacja', 'brzoza', 'dab', 'lipa', 'wierzba']


In [61]:
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [62]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
normalization_8 (Normalizati (None, 224, 224, 3)       7         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1280)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 1281

In [63]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.Poisson(),
    metrics=[keras.metrics.Accuracy()],
)

epochs = 5
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Epoch 1/5
37/37 [==============================] - 68s 2s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: 1.0432 - val_accuracy: 0.0000e+00
Epoch 2/5
37/37 [==============================] - 59s 2s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: 0.8733 - val_accuracy: 0.0000e+00
Epoch 3/5
37/37 [==============================] - 59s 2s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: 0.7443 - val_accuracy: 0.0000e+00
Epoch 4/5
37/37 [==============================] - 59s 2s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: 0.6057 - val_accuracy: 0.0000e+00
Epoch 5/5
37/37 [==============================] - 59s 2s/step - loss: nan - accuracy: 0.0000e+00 - val_loss: 0.6002 - val_accuracy: 0.0000e+00


In [64]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.Poisson(),
    metrics=[keras.metrics.Accuracy()],
)

epochs = 5
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
normalization_8 (Normalizati (None, 224, 224, 3)       7         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_8 ( (None, 1280)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 1281

In [66]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy {}%, loss {}%".format(accuracy*100, loss*100))

8/8 [==============================] - 11s 1s/step - loss: nan - accuracy: 0.0000e+00
Accuracy 0.0%, loss nan%
